In [1]:
import os
import sys

parent = os.path.dirname(os.getcwd())
if parent not in sys.path:
    sys.path.append(parent)
print(parent)
%matplotlib widget

d:\Documents\git\XAI-Cancer-Diagnosis\XAI-Cancer-Diagnosis


# Preencode the BACH cell graphs


In [2]:
from src.transforms.graph_construction.node_embedding import generate_node_embeddings
import torch
from src.transforms.image_processing.filters import glcm

# For each crop-graph, compute the following:
def crop_graph_to_cell_graph(crop_graph,resnet_encoder):
    num_nodes = crop_graph.x.shape[0]
    # Generate GLCM for each cell
    cell_graph = crop_graph.clone()
    cell_graph.x = crop_graph.x.unflatten(1, (3, 64, 64))

    glcm_for_graph = torch.zeros(num_nodes, 50)
    for i, img in enumerate(cell_graph.x):
        # Unflatten the image
        glcm_for_graph[i] = glcm(img, normalize=True)
    
    # Pass the cell images through the resnet encoder
    cell_graph.x = generate_node_embeddings(imgs=cell_graph.x, resnet_encoder=resnet_encoder,
                                            num_neighbours=cell_graph.num_neighbours, cell_types=cell_graph.categories,
                                            glcm=glcm_for_graph.cuda())
    return cell_graph

# TODO: Refactor

In [3]:
from src.datasets.BACH import BACH
from torch_geometric.loader.dataloader import DataLoader

src_folder = os.path.join(
    "C://Users", "aless", "Documents", "FtT", "data", "BACH_TRAIN")

all_data = BACH(src_folder, pre_encoded=False)

KeyboardInterrupt: 

In [ ]:
from src.deep_learning.architectures.cancer_prediction.cell_encoder import CellEncoder

resnet_encoder_path=os.path.join(parent,"model", "CellEncoder.ckpt")
resnet_encoder = CellEncoder.load_from_checkpoint(resnet_encoder_path).eval().cuda()


Using cache found in C:\Users\aless/.cache\torch\hub\pytorch_vision_v0.10.0


In [ ]:
all_data.generate_encoded_graphs(resnet_encoder)

Generating Encoded Graphs: 100%|██████████| 385/385 [03:50<00:00,  1.67it/s]
